In [1]:
%load_ext autoreload
%autoreload 2

In [35]:
#Import the pandas and numpy package
import pandas as pd
import numpy as np

In [36]:
#Load the dataset into dataframe called df
df = pd.read_csv("../data/raw/wfh.csv")

In [4]:
df.head()

,id,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,work_home_actual
0,0,5.962247,40K - 60K,2.119485,8.568058,False,Friday,0.212653,1
1,1,0.535872,40K - 60K,2.357199,5.425382,True,Tuesday,4.927549,0
2,2,1.969519,40K - 60K,2.366849,8.247158,False,Monday,0.520817,1
3,3,2.530410,20K - 40K,2.318722,7.944251,False,Tuesday,0.453649,1
4,4,2.253635,60K+,2.221265,8.884478,True,Thursday,5.695263,1


In [7]:
df.shape

(50000, 9)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          50000 non-null  int64  
 1   distance_from_office        50000 non-null  float64
 2   salary_range                50000 non-null  object 
 3   gas_price_per_litre         50000 non-null  float64
 4   public_transportation_cost  50000 non-null  float64
 5   wfh_prev_workday            50000 non-null  bool   
 6   workday                     50000 non-null  object 
 7   tenure                      50000 non-null  float64
 8   work_home_actual            50000 non-null  int64  
dtypes: bool(1), float64(4), int64(2), object(2)
memory usage: 3.1+ MB


In [9]:
df.describe()

,id,distance_from_office,gas_price_per_litre,public_transportation_cost,tenure,work_home_actual
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,24999.500000,3.929033,2.049616,7.323907,4.600040,0.499580
std,14433.901067,4.079528,0.334385,1.630390,2.301937,0.500005
min,0.000000,0.002210,1.400369,4.003417,0.002253,0.000000
25%,12499.750000,0.897909,1.769163,6.109763,2.797947,0.000000
50%,24999.500000,2.380855,2.189073,8.074422,5.584845,0.000000
75%,37499.250000,5.679015,2.337894,8.627489,6.531917,1.000000
max,49999.000000,19.912896,2.399916,8.998112,6.997167,1.000000


In [37]:
#Create a copy of df and save it into a variable called df_cleaned
df_cleaned = df.copy()

In [38]:
#Extract the column work_home_actual and save it into variable called y
y = df_cleaned["work_home_actual"]

In [42]:
#Import OrdinalEncoder from sklearn.preprocessing
from sklearn.preprocessing import OrdinalEncoder
#Instantiate a OrdinalEncoder with the values from workday column
encoder = OrdinalEncoder()

In [43]:
#Fit and apply the OrdinalEncoder on workday column and replace with the encoded values
df_cleaned["workday"] = encoder.fit_transform(df_cleaned[["workday"]])

In [45]:
#Apply OneHotEncoding on salary_range and save the result back in df_cleaned
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(sparse_output=False, drop='first')

In [31]:
encoded_salary = onehot.fit_transform(df_cleaned[["salary_range"]])

In [33]:
encoded_salary_df = pd.DataFrame(encoded_salary, columns=onehot.get_feature_names_out(["salary_range"]))

In [34]:
df_cleaned = df_cleaned.drop("salary_range", axis=1)
df_cleaned = pd.concat([df_cleaned.reset_index(drop=True), encoded_salary_df.reset_index(drop=True)], axis=1)

In [46]:
#Remove the id column from df_cleaned
df_cleaned = df_cleaned.drop(columns=["id"])

In [47]:
#import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [50]:
#Split the data into training validation and testing sets as Numpy arrays
X = df_cleaned.drop(columns=["work_home_actual"]) 
y = df_cleaned["work_home_actual"]
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

In [51]:
#Print the dimensions of X_train, X_val, X_test
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(30000, 7)
(10000, 7)
(10000, 7)


In [52]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(30000,)
(10000,)
(10000,)


In [53]:
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_val.to_csv('../data/processed/X_val.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_val.to_csv('../data/processed/y_val.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [54]:
#Import the DummyClassifier module from sklearn
from sklearn.dummy import DummyClassifier

In [55]:
#Instantiate the Dummy class into a variable called base_clf and fit it on the training set it
base_clf = DummyClassifier(strategy='most_frequent')
base_clf.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

In [56]:
#Import roc_auc_score from sklearn.metrics
from sklearn.metrics import roc_auc_score

In [57]:
# Display the ROC scores of this baseline model on the training set
y_proba_preds = base_clf.predict_proba(X_train)
roc_auc_score(y_train, y_proba_preds[:, 1])

np.float64(0.5)